In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import TFBertModel,BertTokenizer
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix,classification_report,f1_score
import pandas as pd
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler
early_stoping = EarlyStopping(patience=3)
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import BertModel,AdamWeightDecay, AdamW
import random
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

In [ ]:
train,dev = train_test_split(train,test_size=0.2,random_state=2020)

In [ ]:
train['target'].value_counts()

In [ ]:
sep = '[sep]'
cls = '[cls]'
EPOCHS = 2
BATCH_SIZE = 128
MAX_LENGTH = 32

In [ ]:
#bert_model = BertModel.from_pretrained('bert-base-uncased')
#bert_model.save_pretrained('./save/')

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def build_model(transformer, max_len=512):

    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]

    cls_token = sequence_output[:,0,:]
    # CNN layrers
    out = Dense(3, activation='softmax')(cls_token)
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_masks=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )

    return np.array(enc_di['input_ids'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
bert = build_model(model,MAX_LENGTH)

In [ ]:
train_X = regular_encode(train['question_text'],tokenizer,MAX_LENGTH)
dev_X = regular_encode(dev['question_text'],tokenizer,MAX_LENGTH)
test_X = regular_encode(test['question_text'],tokenizer,MAX_LENGTH)

dev_label = to_categorical(dev['target'],num_classes=2)
train_label = to_categorical(train['target'],num_classes=2)

In [ ]:
bert.fit(train_X,train_label,
         batch_size=128,
         validation_data=(dev_X,dev_label),
         shuffle=True,epochs=EPOCHS,callbacks=[early_stoping])

In [ ]:
pred = bert.predict(test_X).argmax(-1)
qid = test['qid']
test['prediction'] = list(pred)
df = test[['qid','prediction']]
df.to_csv('submission.csv',index=False)